# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

This data represent 17 campaigns. They took place between May 2008 and November 2010. All this represent 79354 contacts.

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [215]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer
from sklearn.metrics import mean_squared_error, accuracy_score, recall_score, precision_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn import set_config
from sklearn.svm import SVC
set_config("figure")
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [216]:
df_full_camp = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [217]:
df_full_camp.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



According tho the description provided, we can see that the values representing missing values, per each column, are:

- job : unknown
- marital : unknown
- education: unknown
- default: unknown
- housing: unknown
- loan: unknown
- poutcome: nonexistent


Besides, some of the data that can be coerced to a different data type are:

- housing and loan
- month and day_of_week



### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [218]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

The Business Objective is to compare the performance of the following classifiers:

- K Nearest Neighbor
- Logistic Regression
- Decision Trees
- Support Vector Machines


In [219]:
df_full_camp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

We notice also that there are not null values. Nevertheless, as we have noticed before, some of the values are "unknown". We will proceed with the dataset as it is since unknown can represent a value to take into account for making business decision.



### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features (columns 1 - 7), prepare the features and target column for modeling with appropriate encoding and transformations.

In [220]:
cols_bank = list(df_full_camp.columns[0:7])
df_bank = pd.concat([df_full_camp[cols_bank],
                     df_full_camp[['y']]
                    ], 
                    axis=1
                   ) 


In [221]:
df_bank.head()

,age,job,marital,education,default,housing,loan,y
0,56,housemaid,married,basic.4y,no,no,no,no
1,57,services,married,high.school,unknown,no,no,no
2,37,services,married,high.school,no,yes,no,no
3,40,admin.,married,basic.6y,no,no,no,no
4,56,services,married,high.school,no,no,yes,no


In [222]:
df_bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        41188 non-null  int64 
 1   job        41188 non-null  object
 2   marital    41188 non-null  object
 3   education  41188 non-null  object
 4   default    41188 non-null  object
 5   housing    41188 non-null  object
 6   loan       41188 non-null  object
 7   y          41188 non-null  object
dtypes: int64(1), object(7)
memory usage: 2.5+ MB


We observe that one column is integer (age). All the rest are of type object.

Let's have a look on the different values contained in each column.

In [223]:
print('unique values in the column AGE: '),print(np.sort(df_bank['age'].unique())),print('\n')
print('unique values in the column JOB: '),print(df_bank['job'].unique()),print('\n')
print('unique values in the column MARITAL: '),print(df_bank['marital'].unique()),print('\n')
print('unique values in the column EDUCATION: '),print(df_bank['education'].unique()),print('\n')
print('unique values in the column DEFAULT: '),print(df_bank['default'].unique()),print('\n')
print('unique values in the column HOUSING: '),print(df_bank['housing'].unique()),print('\n')
print('unique values in the column LOAN: '),print(df_bank['loan'].unique()),print('\n');


unique values in the column AGE: 
[17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40
 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64
 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88
 89 91 92 94 95 98]


unique values in the column JOB: 
['housemaid' 'services' 'admin.' 'blue-collar' 'technician' 'retired'
 'management' 'unemployed' 'self-employed' 'unknown' 'entrepreneur'
 'student']


unique values in the column MARITAL: 
['married' 'single' 'divorced' 'unknown']


unique values in the column EDUCATION: 
['basic.4y' 'high.school' 'basic.6y' 'basic.9y' 'professional.course'
 'unknown' 'university.degree' 'illiterate']


unique values in the column DEFAULT: 
['no' 'unknown' 'yes']


unique values in the column HOUSING: 
['no' 'yes' 'unknown']


unique values in the column LOAN: 
['no' 'yes' 'unknown']




#### Preparation of tansformation and encoding

Since most of the columns are categorical, we will use a OneHotEncoder for all the columsn but the age which is numerical.

In [224]:
transformer = make_column_transformer((OneHotEncoder(drop = 'if_binary'), 
                                       ['job','marital','education','default','housing','loan']
                                      ), 
                                     remainder = StandardScaler())

#### Features and target preparation

In [225]:
X = df_bank.drop(columns=['y'], axis = 1)
y = df_bank['y']

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [226]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

The baseline of our model should be, for the 'no' answer, about **88.7%**

In [227]:
df_bank['y'].value_counts(normalize = True)[0]

0.8873458288821987

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [228]:
logireg_pipe = Pipeline([('transform', transformer),
                         ('lr', LogisticRegression(random_state=42,
                                                   max_iter = 1000
                                                   )
                          )
                         ],
                         verbose=True
                        )


logireg_pipe.fit(X_train, y_train)




[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.1s
[Pipeline] ................ (step 2 of 2) Processing lr, total=   0.3s


Pipeline(steps=[('transform',
                 ColumnTransformer(remainder=StandardScaler(),
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='if_binary'),
                                                  ['job', 'marital',
                                                   'education', 'default',
                                                   'housing', 'loan'])])),
                ('lr', LogisticRegression(max_iter=1000, random_state=42))],
         verbose=True)

### Problem 9: Score the Model

What is the accuracy of your model?

In [229]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [230]:
logireg_train_preds     = logireg_pipe.predict(X_train)
logireg_test_preds     = logireg_pipe.predict(X_test)

logireg_train_accuracy  = accuracy_score(y_train, logireg_train_preds)
logireg_test_accuracy  = accuracy_score(y_test, logireg_test_preds)


The accuracy of the Logistic Regresion model is for the train  set: **88.71%**

In [231]:
logireg_train_accuracy

0.887119225664433

The accuracy of the Logistic Regresion model is for the test set: **88.80%**

In [232]:
logireg_test_accuracy

0.8880256385354958

### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

## KNN algorithm model

In [233]:
knn_pipe = Pipeline([('transform', transformer),
                     ('knn', KNeighborsClassifier())
                    ],
                    verbose=True
                   )


knn_pipe.fit(X_train, y_train)

[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing knn, total=   0.0s


Pipeline(steps=[('transform',
                 ColumnTransformer(remainder=StandardScaler(),
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='if_binary'),
                                                  ['job', 'marital',
                                                   'education', 'default',
                                                   'housing', 'loan'])])),
                ('knn', KNeighborsClassifier())],
         verbose=True)

In [234]:
knn_train_preds     = knn_pipe.predict(X_train)
knn_test_preds     = knn_pipe.predict(X_test)

knn_train_accuracy  = accuracy_score(y_train, knn_train_preds)
knn_test_accuracy  = accuracy_score(y_test, knn_test_preds)

The accuracy of the K Nearest Neighbours model is for the train set: **89.05%**

In [235]:
knn_train_accuracy

0.8905506458191706

The accuracy of the K Nearest Neighbours model is for the test set: **87.66%**

In [236]:
knn_test_accuracy

0.8766631057589589

## Decision Tree model

In [237]:
tree_pipe = Pipeline([('transform', transformer),
                      ('tree', DecisionTreeClassifier())
                     ],
                     verbose=True
                    )


tree_pipe.fit(X_train, y_train)

[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.1s
[Pipeline] .............. (step 2 of 2) Processing tree, total=   0.5s


Pipeline(steps=[('transform',
                 ColumnTransformer(remainder=StandardScaler(),
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='if_binary'),
                                                  ['job', 'marital',
                                                   'education', 'default',
                                                   'housing', 'loan'])])),
                ('tree', DecisionTreeClassifier())],
         verbose=True)

In [238]:
tree_train_preds     = tree_pipe.predict(X_train)
tree_test_preds     = tree_pipe.predict(X_test)

tree_train_accuracy  = accuracy_score(y_train, tree_train_preds)
tree_test_accuracy  = accuracy_score(y_test, tree_test_preds)

The accuracy of the Decision Tree model is for the train set: **91.69%**

In [239]:
tree_train_accuracy

0.9169661066329999

The accuracy of the Decision Tree model is for the test set: **86.37%**

In [240]:
tree_test_accuracy

0.8647178789938818

## SVM model

In [241]:
svm_pipe = Pipeline([('transform', transformer),
                     ('svm', SVC())
                     ],
                     verbose=True
                    )


svm_pipe.fit(X_train, y_train)

[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing svm, total=  38.7s


Pipeline(steps=[('transform',
                 ColumnTransformer(remainder=StandardScaler(),
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='if_binary'),
                                                  ['job', 'marital',
                                                   'education', 'default',
                                                   'housing', 'loan'])])),
                ('svm', SVC())],
         verbose=True)

In [242]:
svm_train_preds     = svm_pipe.predict(X_train)
svm_test_preds     = svm_pipe.predict(X_test)

svm_train_accuracy  = accuracy_score(y_train, svm_train_preds)
svm_test_accuracy  = accuracy_score(y_test, svm_test_preds)

The accuracy of the Support Vector Machine model is for the train set: **88.74%**

In [243]:
svm_train_accuracy

0.8874429445469554

The accuracy of the Support Vector Machine model is for the test set: **88.83%**

In [244]:
svm_test_accuracy

0.8883169855297659

### Summary of the results

In [245]:
pd.DataFrame(data={'Model':['Logistic Regression','KNN','Decision Tree','SVM'],
                   'Train Time':[0.2,0.0,0.4,37.7],
                   'Train Accuracy':[logireg_train_accuracy,knn_train_accuracy,tree_train_accuracy,svm_train_accuracy],
                   'Test Accuracy':[logireg_test_accuracy,knn_test_accuracy,tree_test_accuracy,svm_test_accuracy]
                  } 
            )

,Model,Train Time,Train Accuracy,Test Accuracy
0,Logistic Regression,0.2,0.887119,0.888026
1,KNN,0.0,0.890551,0.876663
2,Decision Tree,0.4,0.916966,0.864718
3,SVM,37.7,0.887443,0.888317


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

# Logistic Regression

In [246]:
logireg_params = {'lr__C': np.linspace(0.01,1,10)}

logireg_grid = GridSearchCV(logireg_pipe, 
                            param_grid=logireg_params
                           )
logireg_grid.fit(X_train, y_train)

[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] ................ (step 2 of 2) Processing lr, total=   0.1s
[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] ................ (step 2 of 2) Processing lr, total=   0.1s
[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] ................ (step 2 of 2) Processing lr, total=   0.1s
[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] ................ (step 2 of 2) Processing lr, total=   0.1s
[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] ................ (step 2 of 2) Processing lr, total=   0.1s
[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] ................ (step 2 of 2) Processing lr, total=   0.1s
[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] ................ (step 2 of 2) Processing lr, total=   0.1s
[Pipel

GridSearchCV(estimator=Pipeline(steps=[('transform',
                                        ColumnTransformer(remainder=StandardScaler(),
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(drop='if_binary'),
                                                                         ['job',
                                                                          'marital',
                                                                          'education',
                                                                          'default',
                                                                          'housing',
                                                                          'loan'])])),
                                       ('lr',
                                        LogisticRegression(max_iter=1000,
                                   

In [247]:
logireg_hyperp = list(logireg_grid.best_params_.values())[0]
logireg_best_test_acc = logireg_grid.score(X_test, y_test)
logireg_best_train_acc = logireg_grid.score(X_train, y_train)

The accuracy of the Logistic Regression model for a Grid Search for the C hyperparameter is, for the train set: **88.71%**

In [248]:
logireg_best_train_acc

0.887119225664433

The accuracy of the Logistic Regression model for a Grid Search for the C hyperparameter is, for the test set: **88.80%**

In [249]:
logireg_best_test_acc

0.8880256385354958

### KNN algorithm model

In [250]:
knn_params = {'knn__n_neighbors': [5,7,10,100,1000]}

knn_grid = GridSearchCV(knn_pipe, 
                        param_grid=knn_params
                       )
knn_grid.fit(X_train, y_train)

[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing knn, total=   0.0s
[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing knn, total=   0.0s
[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing knn, total=   0.0s
[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing knn, total=   0.0s
[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing knn, total=   0.0s
[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing knn, total=   0.0s
[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing knn, total=   0.0s
[Pipel

GridSearchCV(estimator=Pipeline(steps=[('transform',
                                        ColumnTransformer(remainder=StandardScaler(),
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(drop='if_binary'),
                                                                         ['job',
                                                                          'marital',
                                                                          'education',
                                                                          'default',
                                                                          'housing',
                                                                          'loan'])])),
                                       ('knn', KNeighborsClassifier())],
                                verbose=True),
             param_grid={'knn__n_ne

In [251]:
knn_best_hyperp = list(knn_grid.best_params_.values())[0]
knn_best_hyperp


100

In [252]:
knn_best_train_acc = knn_grid.score(X_train, y_train)
knn_best_test_acc = knn_grid.score(X_test, y_test)


The accuracy of the KNN model for a Grid Search for the number of neighbours hyperparameter is, for the train set: **88.69%**

In [253]:
knn_best_train_acc

0.8869573662231718

The accuracy of the KNN model for a Grid Search for the number of neighbours hyperparameter is, for the test set: **88.81%**

In [254]:
knn_best_test_acc

0.8881227542002526

### Decision Tree model

In [255]:
tree_params = {'tree__max_depth': [1,3,5,50,100,500]}

tree_grid = GridSearchCV(tree_pipe, 
                         param_grid=tree_params
                       )
tree_grid.fit(X_train, y_train)

[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] .............. (step 2 of 2) Processing tree, total=   0.0s
[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] .............. (step 2 of 2) Processing tree, total=   0.0s
[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] .............. (step 2 of 2) Processing tree, total=   0.0s
[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] .............. (step 2 of 2) Processing tree, total=   0.0s
[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] .............. (step 2 of 2) Processing tree, total=   0.0s
[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] .............. (step 2 of 2) Processing tree, total=   0.0s
[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] .............. (step 2 of 2) Processing tree, total=   0.0s
[Pipel

GridSearchCV(estimator=Pipeline(steps=[('transform',
                                        ColumnTransformer(remainder=StandardScaler(),
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(drop='if_binary'),
                                                                         ['job',
                                                                          'marital',
                                                                          'education',
                                                                          'default',
                                                                          'housing',
                                                                          'loan'])])),
                                       ('tree', DecisionTreeClassifier())],
                                verbose=True),
             param_grid={'tree__

In [256]:
tree_best_hyperp = list(tree_grid.best_params_.values())[0]
tree_best_hyperp

1

In [257]:
tree_best_train_acc = tree_grid.score(X_train, y_train)
tree_best_test_acc = tree_grid.score(X_test, y_test)


The accuracy of the Decision Tree model for a Grid Search for the max depth hyperparameter is, for the train set: **88.71%**

In [258]:
tree_best_train_acc

0.887119225664433

The accuracy of the Decision Tree model for a Grid Search for the max depth hyperparameter is, for the train set: **88.80%**

In [259]:
tree_best_test_acc

0.8880256385354958

### SVM model

In [260]:
svm_params = {'svm__degree': [0,1,2,3,4,5]}

svm_grid = GridSearchCV(svm_pipe, 
                        param_grid=svm_params
                       )
svm_grid.fit(X_train, y_train)

[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing svm, total=  21.4s
[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing svm, total=  20.5s
[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing svm, total=  21.1s
[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing svm, total=  22.4s
[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing svm, total=  22.4s
[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing svm, total=  21.4s
[Pipeline] ......... (step 1 of 2) Processing transform, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing svm, total=  22.2s
[Pipel

GridSearchCV(estimator=Pipeline(steps=[('transform',
                                        ColumnTransformer(remainder=StandardScaler(),
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(drop='if_binary'),
                                                                         ['job',
                                                                          'marital',
                                                                          'education',
                                                                          'default',
                                                                          'housing',
                                                                          'loan'])])),
                                       ('svm', SVC())],
                                verbose=True),
             param_grid={'svm__degree': [0, 1, 2, 3,

In [261]:
svm_best_hyperp = list(svm_grid.best_params_.values())[0]
svm_best_hyperp

0

In [262]:
svm_best_train_acc = svm_grid.score(X_train, y_train)
svm_best_test_acc = svm_grid.score(X_test, y_test)

The accuracy of the SVM model for a Grid Search for the degree hyperparameter is, for the train set: **88.74%**

In [263]:
svm_best_train_acc

0.8874429445469554

The accuracy of the SVM model for a Grid Search for the degree hyperparameter is, for the test set: **88.83%**

In [264]:
svm_best_test_acc

0.8883169855297659

### Summary of Results

In [265]:
pd.DataFrame(data={'Model':['Logistic Regression','KNN','Decision Tree','SVM'],
                   'Train Time':[0.1,0.0,0.3,21.7],
                   'Train Accuracy':[logireg_best_train_acc,knn_best_train_acc,tree_best_train_acc,svm_best_train_acc],
                   'Test Accuracy':[logireg_best_test_acc,knn_best_test_acc,tree_best_test_acc,svm_best_test_acc],
                   'Hyperparameter tested':['C','n_neighbors','max_depth','degree']
                  } 
            )


,Model,Train Time,Train Accuracy,Test Accuracy,Hyperparameter tested
0,Logistic Regression,0.1,0.887119,0.888026,C
1,KNN,0.0,0.886957,0.888123,n_neighbors
2,Decision Tree,0.3,0.887119,0.888026,max_depth
3,SVM,21.7,0.887443,0.888317,degree


##### Conclusions, next steps  and further recommendations 

For a more accurate analysis, it could be possible to include more features to our analysis data set. Here, we have included just those related to the bank but could include much more. This would eventually increase the computational resources. 

But before including more features, we should wonder the added value to add them. For example, keeping the gender feature could be important if we thinkthat gender can have an impact on the result.

For this analysis we have tuned some hyperparameter for each model, but we could tune more and for a wide range for a deeper analysis. 

In any case, we have seen that, after we tuned our models and we found out the best hyperparameter, the train and test accuracy time are decreased.
